# Florida Electoral Analysis - Data Cleaning

The objectives of this project are to:

1. Identify precincts friendliest to Ds in R counties
2. Identify the counties and precincts with the greatest swing from R to D over the course of multiple elections
3. Identify the counties and precincts with the greatest swing from D to R over the course of multiple elections
4. Identify the counties and precincts surrounded by precincts that swung in an opposite direction -- essentially, identifying where people diverge most from their neighbors

For this analysis, I will be drawing data from the Florida Division of Elections precinct level election results (https://dos.myflorida.com/elections/data-statistics/elections-data/precinct-level-election-results/).

## Step 1: Reformat and Ingest Data

Before we dive into analysis, we have to ingest and clean a **lot** of data, a task to which this entire notebook will be dedicated. The data available from the FDOE is available in .txt format (*yay*) for each county for each election, so we'll have to turn 67 .txt files into one dataframe *for each year we want to investigate*. We'll start with our earliest year of data, which is from the 2012 general election. From here, we can define a standard set of columns into which will:

 * Work for our analytical purposes
 * Be kept consistent across the various years for which we are doing this analysis, as the format the FDOE provides their data in has changed over the years.

In [76]:
import numpy as np
import pandas as pd
import matplotlib as mp
import csv
import os
import glob

To start ingesting the 2012 data, we'll first need to convert all of their provided .txt files into .csv's that we can more readily get into a dataframe. 

To start ingesting and cleaning the data, let's put together a script to:
1. Open each .txt file
2. Use a the tab character as a delimeter for each value
3. Add a semicolon character to the end of each row as a line delimiter.

**Note:** Most of this code is commented out, as it only needs to be run once at the onset of the data cleaning process and I don't want it to run again should I hit 'Run All' in my IDE.

In [77]:
output = "C:\\Users\\Chris Hill\\Documents\\VS Code Repository\\Political Projects\\Florida Analysis\\FL 2012 by Precinct\\CSV Files"
outPath = glob.glob("{0}\*.csv".format(output))

folPath = "C:\\Users\\Chris Hill\\Documents\\VS Code Repository\\Political Projects\\Florida Analysis\\FL 2012 by Precinct\\Text Files"
filPath = glob.glob("{0}\*.txt".format(folPath))

# For future years, turn the below code into a function to be called on the directory of .txt files
# def txt_to_csv(input_path,output_path,fields):

# for file in filPath:
#     with open(file, 'r') as input_file:
#         in_txt = csv.reader(input_file, delimiter='\t')
#         filename = os.path.splitext(os.path.basename(file))[0] + '.csv'
#         stripped = (line.strip() for line in input_file)
#         lines = (line.split("\t") for line in stripped if line)
#         with open(os.path.join(output, filename), 'w') as output_file:
#             fields = ['county_code', 'county_name', 'elec_num', 'elec_date', 'elec_name', 'precinct_id', 'poll_loc', 'total_reg', 'total_reg_r', 'total_reg_d', 'total_reg_other', 'contest_name', 'district', 'contest_code', 'cand_or_issue', 'cand_party', 'cand_id', 'doe_num', 'vote_total']
#             writer = csv.writer(output_file)
#             writer.writerow(fields)
#             writer.writerows(lines)

Now that we have clean(er) .csv files, let's turn them into dataframes and merge 'em all into one big ol' honkin' df.

In [78]:
# merging the files
joined_files = os.path.join(output, "*.csv")
  
# A list of all joined files is returned
joined_list = glob.glob(joined_files)
  
# Finally, the files are joined
df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

We did it! We now have a huge dataframe that has all of our .csv's merged together. Let's take a look at it.

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 688411 entries, 0 to 688410
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   county_code      688411 non-null  object
 1   county_name      688411 non-null  object
 2   elec_num         688411 non-null  int64 
 3   elec_date        688411 non-null  object
 4   elec_name        688411 non-null  object
 5   precinct_id      688411 non-null  object
 6   poll_loc         660772 non-null  object
 7   total_reg        688411 non-null  int64 
 8   total_reg_r      688411 non-null  int64 
 9   total_reg_d      688411 non-null  int64 
 10  total_reg_other  688411 non-null  int64 
 11  contest_name     688411 non-null  object
 12  district         688411 non-null  object
 13  contest_code     688411 non-null  int64 
 14  cand_or_issue    688411 non-null  object
 15  cand_party       627673 non-null  object
 16  cand_id          688411 non-null  int64 
 17  doe_num   

It's a big boi, almost 700k rows! Right off the bat, I see there are some NaN values somewhere in the `poll_loc` and `cand_party` columns -- we'll do a few other things to clean up and format this dataframe first, but that's good to know and we'll be address those shortly.

For the moment, let's just take a look at the shape of it, noting the data in each column:

In [80]:
df

,county_code,county_name,elec_num,elec_date,elec_name,precinct_id,poll_loc,total_reg,total_reg_r,total_reg_d,total_reg_other,contest_name,district,contest_code,cand_or_issue,cand_party,cand_id,doe_num,vote_total
0,ALA,Alachua,9547,11/06/2012,2012 General Election,1,01 First Baptist Church of Waldo,1411,0,0,0,President of the United States,,100000,Romney / Ryan,REP,0,55509,608
1,ALA,Alachua,9547,11/06/2012,2012 General Election,1,01 First Baptist Church of Waldo,1411,0,0,0,President of the United States,,100000,Obama / Biden,DEM,0,55511,381
2,ALA,Alachua,9547,11/06/2012,2012 General Election,1,01 First Baptist Church of Waldo,1411,0,0,0,President of the United States,,100000,Stevens / Link,OBJ,0,59923,1
3,ALA,Alachua,9547,11/06/2012,2012 General Election,1,01 First Baptist Church of Waldo,1411,0,0,0,President of the United States,,100000,Johnson / Gray,LBT,0,59927,6
4,ALA,Alachua,9547,11/06/2012,2012 General Election,1,01 First Baptist Church of Waldo,1411,0,0,0,President of the United States,,100000,"Goode, / Clymer",CPF,0,59941,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688406,WAS,Washington,9547,11/06/2012,2012 General Election,16,COUNTRY OAKS BAPTIST CHURCH,503,0,0,0,Amendment No. 11: ADDITIONAL HOMESTEAD EXEMPTI...,,901100,UnderVotes,,0,902,17
688407,WAS,Washington,9547,11/06/2012,2012 General Election,16,COUNTRY OAKS BAPTIST CHURCH,503,0,0,0,Amendment No. 12: APPOINTMENT OF STUDENT BODY ...,,901200,Yes for Approval,NOP,0,10,121
688408,WAS,Washington,9547,11/06/2012,2012 General Election,16,COUNTRY OAKS BAPTIST CHURCH,503,0,0,0,Amendment No. 12: APPOINTMENT OF STUDENT BODY ...,,901200,No for Rejection,NOP,0,20,197
688409,WAS,Washington,9547,11/06/2012,2012 General Election,16,COUNTRY OAKS BAPTIST CHURCH,503,0,0,0,Amendment No. 12: APPOINTMENT OF STUDENT BODY ...,,901200,OverVotes,,0,901,0


First off, let's just check for dupes:

In [81]:
df.duplicated().sum()

0

No dupes! Great!

Looking back at the dataframe preview, it looks like there might be some columns that don't contain any valuable information that we could drop to clean up our frame:

`total_reg_r`, `total_reg_d`, and `total_reg_other` look like they may not contain any useful values, and `elec_name` and `elec_num` may just be the same for every row and therefor not very useful either. 

Let's take a look:

In [82]:
df['total_reg_r'].unique() # All values are 0, not useful!

array([0], dtype=int64)

In [83]:
df['total_reg_d'].unique() # All values are 0, not useful!

array([0], dtype=int64)

In [84]:
df['elec_num'].unique() # All values are 9547, not useful!

array([9547], dtype=int64)

In [85]:
df['elec_name'].unique() # All values are '2012 General Election', not useful!

array(['2012 General Election'], dtype=object)

In [86]:
df['total_reg_other'].unique() # There are unique values!

array([     0,   2301,   1051,   3048,   2258,    942,   1398,   3753,
         1931,   4456,   4124,   6240,   1482,   3659,   3001,   1682,
         3526,   3249,   3857,   2253,   2606,   2990,   1940,   2763,
         3447,   2529,   2948,   2339,   1651,   1162,   2613,   1458,
         1032,   1402,   1332,   1011,   1704,   1197,   1963,   2072,
         1511,   1735,   2709,   1615,   1397,   2284,   2525,   2042,
          525,   1911,   2368,   2088,   2872,   1875,   1919,   2715,
         4226,   3727,   1862,   3505,   2047,   4269,   4568,   3749,
         4494,   2822,   4653,   2448,   2252,    829,   1648,   3600,
         3708,   1999,   2020,   1572,    699,    527,   3471,   2336,
         2912,    732,    955,   3399,   1113,   2666,   1932,   1769,
         1597,   2264,   2940,   2815,   1497,    834,   1645,   3232,
         1971,   1258,    726,   1846,   2955,   1239,   3514,    549,
         2133,    995,   2439,   1226,   1431,   3354,   3952,   2503,
      

In [87]:
# How many unique values are there?
len(df['total_reg_other'].unique())


245

Looks like we do indeed have some unique values in the `total_reg_other` column -- 245 of them, in fact. 

That's not many overall given the size of our dataframe, but let's make a new dataframe with just those rows to see what they look like:

In [88]:
df_reg_other = df[df['total_reg_other'] != 0]
df_reg_other.head()

,county_code,county_name,elec_num,elec_date,elec_name,precinct_id,poll_loc,total_reg,total_reg_r,total_reg_d,total_reg_other,contest_name,district,contest_code,cand_or_issue,cand_party,cand_id,doe_num,vote_total
606479,POL,Polk,9547,11/06/2012,2012 General Election,101,Precinct 101,2301,0,0,2301,President of the United States,,100000,Times Over Voted,NaN,0,901,5
606480,POL,Polk,9547,11/06/2012,2012 General Election,101,Precinct 101,2301,0,0,2301,President of the United States,,100000,Romney / Ryan,REP,0,55509,597
606481,POL,Polk,9547,11/06/2012,2012 General Election,101,Precinct 101,2301,0,0,2301,President of the United States,,100000,Obama / Biden,DEM,0,55511,359
606482,POL,Polk,9547,11/06/2012,2012 General Election,101,Precinct 101,2301,0,0,2301,President of the United States,,100000,Stevens / Link,OBJ,0,59923,1
606483,POL,Polk,9547,11/06/2012,2012 General Election,101,Precinct 101,2301,0,0,2301,President of the United States,,100000,Johnson / Gray,LBT,0,59927,3


It's not clear to me what the values in `total_reg_other` signify, especially given that the other two columns `total_reg_r` and `total_reg_d` are essentially empty, but it looks like it may just mirror the value in the `total_reg` column for a given precinct. 

Let's find out by creating a new dataframe (`df_reg_temp`) that should only be populated with rows wherein `total_reg_other` is not equal to `total_reg`:

In [97]:
df_reg_temp = df_reg_other[df_reg_other['total_reg_other'] != df_reg_other['total_reg']]
df_reg_temp.head()

,county_code,county_name,elec_num,elec_date,elec_name,precinct_id,poll_loc,total_reg,total_reg_r,total_reg_d,total_reg_other,contest_name,district,contest_code,cand_or_issue,cand_party,cand_id,doe_num,vote_total


And it's empty! We can safetly say there is no unique data in the `total_reg_other` column that isn't already in `total_reg` and add it to our list of columns to drop.

In [90]:
df = df.drop(columns=['total_reg_r', 'total_reg_d', 'total_reg_other', 'elec_num', 'elec_name'])
df.head()

,county_code,county_name,elec_date,precinct_id,poll_loc,total_reg,contest_name,district,contest_code,cand_or_issue,cand_party,cand_id,doe_num,vote_total
0,ALA,Alachua,11/06/2012,1,01 First Baptist Church of Waldo,1411,President of the United States,,100000,Romney / Ryan,REP,0,55509,608
1,ALA,Alachua,11/06/2012,1,01 First Baptist Church of Waldo,1411,President of the United States,,100000,Obama / Biden,DEM,0,55511,381
2,ALA,Alachua,11/06/2012,1,01 First Baptist Church of Waldo,1411,President of the United States,,100000,Stevens / Link,OBJ,0,59923,1
3,ALA,Alachua,11/06/2012,1,01 First Baptist Church of Waldo,1411,President of the United States,,100000,Johnson / Gray,LBT,0,59927,6
4,ALA,Alachua,11/06/2012,1,01 First Baptist Church of Waldo,1411,President of the United States,,100000,"Goode, / Clymer",CPF,0,59941,1


Great! Five superfluous columns removed. 

We still have a lot of clutter in here; each row is currently the vote count per precinct per race per candidate. Let's clean the dataframe to only include pertinent federal and state leg races:

In [91]:
df_partisan_races_2012 = df[df.contest_name.isin(['President of the United States',\
                                'United States Senator',\
                                'U.S. Representative',\
                                'State Senator',\
                                'State Representative'])]\
                                .reset_index()

And let's check it out:

In [92]:
df_partisan_races_2012.head()

,index,county_code,county_name,elec_date,precinct_id,poll_loc,total_reg,contest_name,district,contest_code,cand_or_issue,cand_party,cand_id,doe_num,vote_total
0,0,ALA,Alachua,11/06/2012,1,01 First Baptist Church of Waldo,1411,President of the United States,,100000,Romney / Ryan,REP,0,55509,608
1,1,ALA,Alachua,11/06/2012,1,01 First Baptist Church of Waldo,1411,President of the United States,,100000,Obama / Biden,DEM,0,55511,381
2,2,ALA,Alachua,11/06/2012,1,01 First Baptist Church of Waldo,1411,President of the United States,,100000,Stevens / Link,OBJ,0,59923,1
3,3,ALA,Alachua,11/06/2012,1,01 First Baptist Church of Waldo,1411,President of the United States,,100000,Johnson / Gray,LBT,0,59927,6
4,4,ALA,Alachua,11/06/2012,1,01 First Baptist Church of Waldo,1411,President of the United States,,100000,"Goode, / Clymer",CPF,0,59941,1


In [93]:
df_partisan_races_2012.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167915 entries, 0 to 167914
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   index          167915 non-null  int64 
 1   county_code    167915 non-null  object
 2   county_name    167915 non-null  object
 3   elec_date      167915 non-null  object
 4   precinct_id    167915 non-null  object
 5   poll_loc       160880 non-null  object
 6   total_reg      167915 non-null  int64 
 7   contest_name   167915 non-null  object
 8   district       167915 non-null  object
 9   contest_code   167915 non-null  int64 
 10  cand_or_issue  167915 non-null  object
 11  cand_party     159171 non-null  object
 12  cand_id        167915 non-null  int64 
 13  doe_num        167915 non-null  int64 
 14  vote_total     167915 non-null  int64 
dtypes: int64(6), object(9)
memory usage: 19.2+ MB


# Starting with Congressional Races

Great! We've narrowed our df down to about 168,000 rows instead of almost 700,000 -- that should be considerably easier to work with. Let's double check to make sure all 27 congressional districts are represented:

In [94]:
district_list = df_partisan_races_2012.district[df_partisan_races_2012.contest_name =='U.S. Representative']
print(district_list)

len(district_list.unique())

14         District 3
15         District 3
16         District 3
17         District 3
38         District 3
             ...     
167901     District 2
167902     District 2
167903     District 2
167904     District 2
167905     District 2
Name: district, Length: 24783, dtype: object


25

Hmmm, it looks like we only have 25 of the 27 districts. Which districts are missing?

In [95]:
district_list.unique()

array([' District 3', ' District 5', ' District 4', ' District 2',
       ' District 8', ' District 22', ' District 21', ' District 20',
       ' District 23', ' District 25', ' District 17', ' District 11',
       ' District 19', ' District 27', ' District 26', ' District 1',
       ' District 6', ' District 14', ' District 12', ' District 10',
       ' District 16', ' District 18', ' District 9', ' District 7',
       ' District 13'], dtype=object)

CD15 and CD24 appear to be missing from our dataframe. After some investigation, I realize this is because both districts had congressional candidates that ran unopposed in the general (R in CD15 and D in CD24). No votes were cast for these candidates in the general election, so it makes sense that none were tallied!

In [96]:
pct_to_cd_2012 = 

SyntaxError: invalid syntax (813989006.py, line 1)